In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as gbm
from sklearn.svm import SVC

/Users/sarvesh/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Getting Data
train = pd.read_csv("train_set.csv")
train_labels = pd.read_csv("train_set_labels.csv")

train = pd.merge(train, train_labels, on='id')


In [3]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [4]:
columns = list(set(list(train.columns)[1:-1]) - set(['num_private', 'basin', 
                                                     'lga', 'region', 'funder', 
                                                     'installer', 'scheme_name',
                                                    'scheme_management']))
X = train[columns]
Y = train[list(train.columns)[-1]]

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_extraction import DictVectorizer

from sklearn.preprocessing import LabelEncoder

X_p = X
lb_make = LabelEncoder()
for column in columns:
    if X[column].dtype not in ['int64', 'float64']:
        X_p[column] = X[column].astype('category')
        X_p[column] = lb_make.fit_transform(X[column].fillna(method='ffill'))
        
Y_p = lb_make.fit_transform(Y)
# ohe = OneHotEncoder()
# ohe.fit(X['funder'])
# ohe.transform(X['funder']).head()
# X.head()
# cols = list(set(columns) - set(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'region_code', 'district_code', 'population']))
# X_p = pd.get_dummies(X)


/Users/sarvesh/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/sarvesh/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
# from sklearn.feature_selection import SelectKBest
# skb = SelectKBest(k = 10000)
# skb.fit(X_p, Y)
# X_p = skb.transform(X_p, Y)
X_p.shape
X_p.head()

,waterpoint_type,ward,source,date_recorded,payment,quality_group,latitude,permit,extraction_type_class,waterpoint_type_group,...,extraction_type_group,district_code,construction_year,public_meeting,longitude,management,extraction_type,quantity_group,amount_tsh,management_group
0,1,1426,8,47,2,2,-9.856322,0,0,1,...,1,5,1999,1,34.938093,7,3,1,6000.0,4
1,1,1576,5,309,0,2,-2.147466,1,0,1,...,1,2,2010,1,34.698766,11,3,2,0.0,4
2,2,1624,0,300,4,2,-3.821329,1,0,1,...,1,4,2009,1,37.460664,7,3,1,25.0,4
3,2,1571,3,272,0,2,-11.155298,1,5,1,...,10,63,1986,1,38.486161,7,14,0,0.0,4
4,1,1687,5,104,0,2,-1.825359,1,0,1,...,1,1,0,1,31.130847,1,3,3,0.0,1


In [10]:
# TODO
pca = PCA(n_components=18)
X_p = pca.fit_transform(X_p, Y)
# y = pca.fit_transform(Y_p)
# y.shape
X_p.shape

(59400, 18)

In [11]:
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

xtrain, xtest, ytrain, ytest = train_test_split(X_p, Y_p, test_size=0.2)


In [21]:
x = XGBClassifier(max_depth=18, learning_rate=0.03, n_estimators=400, gamma=3, nthread=6)
x.fit(xtrain, ytrain)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=3, learning_rate=0.03, max_delta_step=0, max_depth=18,
       min_child_weight=1, missing=None, n_estimators=400, nthread=6,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [22]:
pred = x.predict(xtrain)

print(classification_report(ytrain, pred))

pred = x.predict(xtest)


print(classification_report(ytest, pred))

             precision    recall  f1-score   support

          0       0.87      0.97      0.92     25734
          1       0.92      0.49      0.64      3490
          2       0.94      0.87      0.90     18296

avg / total       0.90      0.90      0.89     47520

             precision    recall  f1-score   support

          0       0.77      0.89      0.83      6525
          1       0.58      0.23      0.32       827
          2       0.81      0.72      0.76      4528

avg / total       0.77      0.78      0.77     11880



In [23]:
from sklearn.metrics import accuracy_score, precision_score

print(accuracy_score(ytest, pred))
print(precision_score(ytest, pred, average='weighted'))

0.780134680135
0.773764662744


In [43]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(boosting_type='dart', num_leaves=173, max_depth=14, learning_rate=0.2, objective='multiclass',
                 n_estimators=350, n_jobs=6)
lgbm.fit(xtrain, ytrain)

LGBMClassifier(boosting_type='dart', colsample_bytree=1.0, learning_rate=0.2,
        max_bin=255, max_depth=14, min_child_samples=10,
        min_child_weight=5, min_split_gain=0.0, n_estimators=350, n_jobs=6,
        num_leaves=173, objective='multiclass', random_state=0,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=50000, subsample_freq=1)

In [44]:
lgbmpred = lgbm.predict(xtrain)

print(classification_report(ytrain, lgbmpred))

lgbmpred = lgbm.predict(xtest)

print(classification_report(ytest, lgbmpred))

             precision    recall  f1-score   support

          0       0.93      0.99      0.96     25734
          1       0.97      0.82      0.89      3490
          2       0.98      0.92      0.95     18296

avg / total       0.95      0.95      0.95     47520

             precision    recall  f1-score   support

          0       0.78      0.89      0.83      6525
          1       0.54      0.25      0.34       827
          2       0.81      0.73      0.77      4528

avg / total       0.78      0.78      0.77     11880



In [45]:
print(accuracy_score(ytest, lgbmpred))
print(precision_score(ytest, lgbmpred, average='weighted'))

0.782491582492
0.775157132339


In [ ]:
svm = SVC(kernel='poly', degree=2, verbose=True)

svm.fit(xtrain, ytrain)

[LibSVM]

In [23]:
svmpred = svm.predict(xtrain)

print(classification_report(ytrain, svmpred))

svmpred = svm.predict(xtest)

print(classification_report(ytest, svmpred))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00     25769
          1       0.00      0.00      0.00      3471
          2       0.38      1.00      0.56     18280

avg / total       0.15      0.38      0.21     47520

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      6490
          1       0.00      0.00      0.00       846
          2       0.38      1.00      0.55      4544

avg / total       0.15      0.38      0.21     11880



/Users/sarvesh/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
print(accuracy_score(ytest, svmpred))
print(precision_score(ytest, svmpred, average='weighted'))